In [3]:
import pandas as pd
from modele_db import *
from Analiza_pianek import analiza_pianek as ap

analiza = ap.analiza
komplety_pianek = ap.komplety_pianek
pda = ap.pda
zam_pianki = ap.zam_pianki

In [4]:
nr_pw = "24/15"
pw = pd.read_excel("Z:/450. PLANISTA - ZAOPATRZENIE/ZAM_PIANKI.xlsx", sheet_name="PW")
pw= pw[pw.PW.str.contains(nr_pw)].merge(komplety_pianek[["KOD", "obj"]], how="left", on="KOD")
pw["SPAKOWANE_M3"] = pw.ILOSC * pw.obj

print("OBIĘTOSC PACZEK (Z OWATAMI)")
sum_obj = 0
for p in pda:
  p_obj = (analiza[p]*analiza.obj).sum()
  sum_obj += p_obj
  print(f"{p}: {p_obj:.0f}m3")


print(f"WST: {(analiza.WST*analiza.obj).sum():.0f}m3")
print(f"Objetość wszystkich paczek: {sum_obj + (analiza.WST*analiza.obj).sum():.0f}m3")
print(f"PW {nr_pw} spakowano: {pw.SPAKOWANE_M3.sum():.0f}M3")

OBIĘTOSC PACZEK (Z OWATAMI)
05/01: 173m3
06/01: 184m3
07/01: 176m3
08/01: 176m3
09/01: 168m3
WST: 115m3
Objetość wszystkich paczek: 991m3
PW 24/15 spakowano: 195M3


In [5]:
pianki_niespakowane = zam_pianki[zam_pianki.dostarczono == 2]

In [6]:
br = ap.Braki(WOLNE="SALDO")[0] #USUNAC PROBLEM Z DUBLUJACYMI SIE ZAMOWIANIAMI(JEDNO DOSTARCZONE DRUGIE W DRODZE TWORZĄ SIĘ DWIE POZYCJE)
br#.to_excel("BRAKI.xlsx")
# br#.sort_values(by=["GRUPA", "OPIS"])

,OPIS,WOLNE_SALDO,SALDO,PACZKA,DATA_KOMPLETACJI,ZAMOWIONE,UWAGI,GRUPA
0,"AMALFI 2,5",-9.0,3.0,06/01,2024-03-06,20.0,"PIANKI ZAMOWIONE P, , ,",2
1,AMALFI NW = WN,-21.0,8.0,06/01,2024-03-06,45.0,"PIANKI ZAMOWIONE P, , ,",2
2,AMALFI [RS = RS],-19.0,2.0,06/01,2024-03-06,20.0,"PIANKI ZAMOWIONE P, , ,",2
3,AMALFI NW MEMORY,-4.0,16.0,09/01,2024-03-27,65.0,CZEKA NA SPAKOWANIE,4
4,AMALFI NW MEMORY,-4.0,16.0,09/01,2024-03-27,65.0,"PIANKI ZAMOWIONE P, , ,",2
5,AMALFI P102x80,-2.0,1.0,08/01,2024-03-20,0.0,...,1
6,CALYPSO P102x102,-1.0,1.0,09/01,2024-03-27,0.0,...,1
7,"EXTREME 1,5s",-3.0,0.0,09/01,2024-03-27,0.0,...,1
8,EXTREME [1s,-4.0,1.0,06/01,2024-03-06,0.0,...,1
9,"EXTREME [1,5s",-2.0,0.0,09/01,2024-03-27,0.0,...,1


In [7]:
ap.Zagrozone(WOLNE="SALDO")

PONIZEJ MIN: 76 POZYCJE
PONIZEJ MIN NIE ZAMOWIONE: 52 POZYCJE


In [8]:
from modele_pianek import *


In [9]:
ar=[ap.Analiza_Rodziny(AMALFI_P,2,1),
    ap.Analiza_Rodziny(AVANT_P,1,1),
    ap.Analiza_Rodziny(CALYPSO,3,2),
    ap.Analiza_Rodziny(COCO_PV,4),
    ap.Analiza_Rodziny(DIVA,2,10),
    ap.Analiza_Rodziny(DUO,3),
    ap.Analiza_Rodziny(ELIXIR,1,1),
    # ap.Analiza_Rodziny(EXTREME,3), #ZROBIC KLASE
    ap.Analiza_Rodziny(GREY,4),
    ap.Analiza_Rodziny(HUDSON,3,2),
    ap.Analiza_Rodziny(HORIZON_PV,1,1),
    # ap.Analiza_Rodziny(KELLY), #POPRAWIC KLASE!!!!
    ap.Analiza_Rodziny(LENOX,4),
    ap.Analiza_Rodziny(MAXWELL_P,2,2),
    # ap.Analiza_Rodziny(MYSTIC,3), #NIE/ZROBIC KLASE?
    ap.Analiza_Rodziny(ONYX,3,2),
    ap.Analiza_Rodziny(OVAL_P,4),
    ap.Analiza_Rodziny(OXYGEN_P,2),
    ap.Analiza_Rodziny(RITZ,3),
    ap.Analiza_Rodziny(WILLOW,2,2),
    ap.Analiza_Rodziny(REVERSO,2,1),
    ap.Analiza_Rodziny(SPECTRA,3),
    ap.Analiza_Rodziny(SAMOA,3,2),
    ap.Analiza_Rodziny(UNO,4),
    ap.Analiza_Rodziny(STONE,1,1)]

ar_podsum = pd.DataFrame([x.Raport() for x in ar])

In [10]:
ard = {a.MODEL: a for a in ar}

In [11]:
ar_podsum.sort_values(by=["GRUPA", "OBJ_BRYL_DO_ZAM_DO_OBJ_MAX"], ascending=[True,False])

,GRUPA,MODEL,POZYCJE_ZAGROZONE,BRAKI,ILOSC_BRAKOW,BRYL_DO_ZAMOWIENIA,OBJ_BRYL_DO_ZAM_DO_OBJ_MAX,OBJ_CIECH,OBJ_VITA,OBJ_PIANPOL
6,1,ELIXIR,2,0,0.0,143.0,0.392547,35.903556,26.355456,0.000000
9,1,HORIZON,6,0,0.0,197.0,0.167574,0.000000,46.682635,9.845820
1,1,AVANT,10,0,0.0,147.0,0.161025,0.000000,0.000000,38.590499
21,1,STONE,6,1,4.0,109.0,0.034404,0.000000,5.393110,0.000000
4,2,DIVA,8,0,0.0,129.0,0.548052,42.661514,0.000000,0.000000
0,2,AMALFI,8,5,55.0,211.0,0.446593,0.000000,0.000000,89.612171
17,2,REVERSO,4,1,3.0,307.0,0.389301,35.888853,27.398147,0.000000
16,2,WILLOW,8,3,19.0,75.0,0.167063,15.268644,9.047669,0.000000
14,2,OXYGEN,0,0,0.0,4.0,0.011136,0.000000,0.000000,0.698308
11,2,MAXWELL,2,0,0.0,1.0,0.006704,0.000000,0.000000,0.502864


In [ ]:
podsumowanie_VOL = ar_podsum[["OBJ_CIECH",	"OBJ_VITA",	"OBJ_PIANPOL"]].sum()
podsumowanie_VOL["RAZEM"] = podsumowanie_VOL.sum()
podsumowanie_VOL